## 1. Interopérabilité entre Python et C

### 1.1 Utilisation de Python API C/C++

__Voici la bibliothèque originalement écrite en C__

In [ ]:
%%writefile c:\temp\Arithmetique\arithemtique.c

#include <Python.h>

/*Notre bibliothèque*/
double caddition(double x,double y)
{
    return x + y;
}

double csoustraction(double x,double y)
{
    return x + y;
}

double cmultiplication(double x,double y)
{
    return x * y;
}

double cdivision(double x,double y)
{
    return x / y;
}


__Il faut tout d'abord écrire les fonctions eveloppe pour chaque foncton de la bibliothèque__

In [ ]:
%%loadfile  c:\temp\Arithmetique\arithemtique.c
%%writefile c:\temp\Arithmetique\arithemtique.c
/*Les fonctions enveloppe*/
static PyObject* CAddition(PyObject *self, PyObject *args)
{
    double a,b;
    double sts=0;

    if (!PyArg_ParseTuple(args, "dd", &a,&b)) return NULL;
    sts = caddition(a,b);
    
    if (b == 0) {
        return NULL;
    }
     
    return PyLong_FromLong(sts);
}

static PyObject* CMultiplication(PyObject *self, PyObject *args)
{
    double a,b;
    double sts=0;

    if (!PyArg_ParseTuple(args, "dd", &a,&b)) return NULL;
    sts = cmultiplication(a,b);
    
    if (b == 0) {
        return NULL;
    }
     
    return PyLong_FromLong(sts);
}



static PyObject* CDivision(PyObject *self, PyObject *args)
{
    double a,b;
    double sts=0;

    if (!PyArg_ParseTuple(args, "dd", &a,&b)) return NULL;
    sts = cdivision(a,b);
    
    if (b == 0) {
        return NULL;
    }
     
    return PyLong_FromLong(sts);
}

static PyObject* version(PyObject *self)
{
   return Py_BuildValue("s","Version 1.0");
}


__En suite, il faut écrire la fonction table qui fournit des méta données sur les fonctions enveloppes, il ne faut pas oublier de finir avec le terminateur  {NULL, NULL, 0, NULL}__ 

In [ ]:
%%loadfile  c:\temp\Arithmetique\arithemtique.c
%%writefile c:\temp\Arithmetique\arithemtique.c
static PyMethodDef methods[] = 
{
    {"division", CDivision, METH_VARARGS,"Effectuer une division."},
    {"version", (PyCFunction)version, METH_NOARGS,"Afficher la version."},
    {NULL, NULL, 0, NULL} 
};


__Par la suite, il faut créer une structure qui représente le module qui sera ensuite invoqué dans notre code Python__

In [ ]:
%%loadfile  c:\temp\Arithmetique\arithemtique.c
%%writefile c:\temp\Arithmetique\arithemtique.c
    
static struct PyModuleDef arthmodule = {
    PyModuleDef_HEAD_INIT,
    "arithmetique",   /* name of module */
    "arithmetique_doc", /* module documentation, may be NULL */
    -1,       /* size of per-interpreter state of the module,
                 or -1 if the module keeps state in global variables. */
    methods
};


__Finalement, il faut écrire la fonction qui crée le module__

In [ ]:
%%loadfile  c:\temp\Arithmetique\arithemtique.c
%%writefile c:\temp\Arithmetique\arithemtique.c
PyMODINIT_FUNC PyInit_arithmetique(void)
{
  return PyModule_Create(&arthmodule);
}

__Il faut passer à la deuxième étape, qui est la création du fichier setup.py qui permet de générer le dossier Build qui contient du code compilé près à l'installation dans Python__

__Remarque__:Le fichier doit figurer dans le même répertoire que le code enveloppe

In [ ]:
%%writefile c:\temp\Arithmetique\setup.py
from distutils.core import setup, Extension

module = Extension('arithmetique', sources=['arithmetique.c'])
setup(name='ArithmetiquePackage',version='1.0',description='test de arithmétique',ext_modules=[module])


__Une fois c'est fait, il faut lancer les deux commandes__

1. python3 setup.py build <br>
2. sudo python3 setup.py install

La commande __python3 setup.py build__ genère un dossier build qui contient le code compilé

La commande __sudo python3 setup.py install__ installe le code dans python

### 1.1 Utilisation de cython

__Comment faire?__

1. Installer cython

2. __cython__ est exprimé en un language hybride proche de __Python__

3. La commande __cython__  genère l'interface en termes de C selon __Python API C__ à partir du code Python

4. Ensuite génére les bibliothèques en __.o__ et __.so__

5. Le code pourra ensuite être partagé entre __C__ et __Python__


Il existe trois manières pour intéragir avec  __cython__ 
et  C à partir de  __Python__ <br>
1. Utiliser le module __Cython__ pour le cas de __Jupyter Notebook__ <br>
2. Installer le code __cython__ représenté sous forme de fichiers __.pyx__ dans __Python__ à l'aide d'un fichier   __setup.py__ et __distutils__ <br>
3. Utiliser le module __pyximport__ pour importer du code prersité dans les fichiers __.pyx__ dynmiquement dans le code __Python__ 

Pour consulter la documentation de cython cliquer [ici](https://cython.org/)


### 1.1.1 Test de Cython sous Jupyter

Pour tester du cython au niveau de jupyter, il faut suivre ces étapes

__Remarque__: Il faut avoir un compilateur c/c++ installé au niveau de la machine
    sinon, l'erreur affichant un fichier bat maquant va être levée

In [18]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [ ]:
%%cython

cdef int factoriel(int n):
        if (n==0):
            return 1
        elif(n==1):
            return 1
        else:   
            return n*factoriel(n-1) 

Pour ajouter les déails d'execution du code ajouterl'option --annotate ou -a à la méthode magique %%cython

In [ ]:
%%cython --annotate

cdef int factoriel(int n):
        if (n==0):
            return 1
        elif(n==1):
            return 1
        else:   
            return n*factoriel(n-1) 

Cliquez ensuite les + pour développer les sections qui contiennent la génération automatique du code en C